In [ ]:
import copy

import colicoords as cc
import h5py
import numpy as np
import scipy as sp
import skimage as sk
import skimage.measure
import skimage.morphology

cc.config.cfg.IMG_PIXELSIZE = 212

from matplotlib import pyplot as plt

In [ ]:
with h5py.File(
    "/home/de64/scratch/de64/sync_folder/2021-06-14_lDE20_biofloat_fullrun_1/Growth_Division/fluorsegmentation/segmentation_1000.hdf5",
    "r",
) as infile:
    img = infile["data"][:]
with h5py.File(
    "/home/de64/scratch/de64/sync_folder/2021-06-14_lDE20_biofloat_fullrun_1/Growth_Division/kymograph/kymograph_1000.hdf5",
    "r",
) as infile:
    fluo_img = infile["mCherry"][:]

In [ ]:
example_mask = img[0, 0] == 5
example_label = img[5, 110]
example_kymo = img[0]
example_fluor = fluo_img[0]
plt.imshow(example_label)

In [ ]:
plt.imshow(example_label)

In [ ]:
np.max(example_label)

In [ ]:
coords = sk.measure.find_contours(example_label, level=0)[0]

In [ ]:
coords

In [ ]:
test_rp = sk.measure.regionprops(example_mask.astype(int))

In [ ]:
rps = sk.measure.regionprops(example_label)

In [ ]:
def get_rot_mat(theta):
    R = np.array([[np.cos(theta), -np.sin(theta)], [np.sin(theta), np.cos(theta)]])
    return R


def center_and_rotate(label_arr):
    rps = sk.measure.regionprops(label_arr)
    coord_list = []

    for i, rp in enumerate(rps):
        R = get_rot_mat(rp.orientation)
        centroid = np.array(list(rp.centroid))
        img = label_arr == (i + 1)
        coords = sk.measure.find_contours(img, level=0)[0]
        centered_coords = coords - centroid
        rotated_coords = (R @ centered_coords.T).T
        reflected_coords = copy.copy(rotated_coords)
        reflected_coords[:, 1] = -reflected_coords[:, 1]
        all_coords = np.concatenate([rotated_coords, reflected_coords], axis=0)
        half_coords = all_coords[all_coords[:, 1] > 0.0]
        half_coords = half_coords[np.argsort(half_coords[:, 0])]
        coord_list.append(half_coords)

    return coord_list


def get_cell_dimensions(coord_list, min_dist_integral_start=0.3):
    cell_dimensions = []

    for coords in coord_list:
        cell_dimension_dict = {}

        y_coords = coords[:, 0]
        x_coords = coords[:, 1]

        no_cap_mask = x_coords > min_dist_integral_start
        filtered_y_coords = y_coords[no_cap_mask]
        filtered_x_coords = x_coords[no_cap_mask]

        spline_fit = sp.interpolate.UnivariateSpline(y_coords, x_coords, k=4)
        a, b = np.min(filtered_y_coords), np.max(filtered_y_coords)
        x = np.linspace(a, b, num=int((b - a)))
        f_x = spline_fit(x)

        vol = np.pi * sp.integrate.simpson(f_x**2, x)
        SA = 2 * np.pi * spline_fit.integral(a, b)
        width = 2 * np.max(f_x)
        height = np.max(y_coords) - np.min(y_coords)

        cell_dimension_dict = {
            "Volume": vol,
            "Surface Area": SA,
            "Width": width,
            "Height": height,
        }

        cell_dimensions.append(cell_dimension_dict)

    return cell_dimensions

In [ ]:
coord_list = center_and_rotate(example_label)
get_cell_dimensions(coord_list)

In [ ]:
plt.imshow(example_label)

In [ ]:
example_label

In [ ]:
idx = 5
min_dist_integral_start = 0.3
y_coords = coord_list[idx][:, 0]
x_coords = coord_list[idx][:, 1]

no_cap_mask = x_coords > min_dist_integral_start
filtered_y_coords = y_coords[no_cap_mask]
filtered_x_coords = x_coords[no_cap_mask]

spline_fit = sp.interpolate.UnivariateSpline(y_coords, x_coords, k=4)
a, b = np.min(filtered_y_coords), np.max(filtered_y_coords)
x = np.linspace(a, b, num=int((b - a)))
f_x = spline_fit(x)

vol = np.pi * sp.integrate.simpson(f_x**2, x)
SA = 2 * np.pi * spline_fit.integral(a, b)
width = 2 * np.max(f_x)
height = np.max(y_coords) - np.min(y_coords)

print(width)
print(height)
print((SA * (0.2125**2)) / (vol * (0.2125**3)))
plt.figure(figsize=(6, 2))
plt.scatter(y_coords, x_coords)
plt.ylim(0, 4)
plt.plot(x, spline_fit(x), "g", lw=3)

In [ ]:
print((SA * (0.2125**2)) / (vol * (0.2125**3)))
plt.figure(figsize=(4, 1))
plt.scatter(y_coords, x_coords)
plt.ylim(0, 4)
plt.plot(x, test(x), "g", lw=3)

In [ ]:
SA

In [ ]:
vol

In [ ]:
plt.figure(figsize=(4, 1))
plt.scatter(y_coords, x_coords)
plt.ylim(0, 4)
plt.plot(x, test(x), "g", lw=3)

In [ ]:
sp.signal.qspline1d(signal, lamb=0.0)

In [ ]:
xs = np.linspace(-12, 12, 1000)
plt.plot(xs, test(xs), "g", lw=3)

In [ ]:
filtered_y = sp.signal.medfilt(coord_list[0][:, 0], kernel_size=(5,))
filtered_x = sp.signal.medfilt(coord_list[0][:, 1], kernel_size=(5,))

In [ ]:
plt.figure(figsize=(5, 1))
plt.plot(filtered_y, filtered_x)
plt.ylim(0, 4)

In [ ]:
plt.figure(figsize=(20, 20))
plt.scatter(coord_list[0][:, 1], coord_list[0][:, 0])
plt.show()

In [ ]:
plt.figure(figsize=(20, 20))
plt.imshow(example_mask)
plt.plot(coords[:, 1], coords[:, 0], "-r", linewidth=2)
plt.show()

In [ ]:
example_kymo.shape

In [ ]:
plt.imshow(example_kymo[80], cmap="gray_r")

In [ ]:
def get_cell_from_mask(mask):
    data = cc.Data()
    data.add_data(example_mask.astype(int), "binary")
    orient = data.binary_img.orientation
    data = data.rotate(orient)

    cell = cc.Cell(
        data,
    )
    cell.optimize()
    return cell

In [ ]:
data = cc.Data()
data.add_data(example_mask.astype(int), "binary")
orient = data.binary_img.orientation
oriented_data = data.rotate(orient)

In [ ]:
plt.imshow(oriented_data.binary_img)

In [ ]:
cell = cc.Cell(
    oriented_data,
)
cell.optimize()
cp = cc.CellPlot(cell)

In [ ]:
fig, axes = plt.subplots(1, 1, figsize=(6, 2))
cp.imshow("binary", ax=axes)
cp.plot_outline(ax=axes)
cp.plot_midline(ax=axes)

### implement batch processing of cells

In [ ]:
plt.imshow(example_fluor[0])

In [ ]:
test_rps = sk.measure.regionprops(example_kymo[0], intensity_image=example_fluor[0])

In [ ]:
plt.imshow(test_rps[0].intensity_image)

In [ ]:
test_rps[0].intensity_image.shape

In [ ]:
rps = sk.measure.regionprops(example_kymo[0])

In [ ]:
def get_isolated_cells(img, padding=4):
    rps = sk.measure.regionprops(img)
    mask_stack = [rp.image for rp in rps]
    mask_stack = [
        np.pad(mask, pad_width=padding, mode="constant", constant_values=False)
        for mask in mask_stack
    ]
    return mask_stack


def get_cell_from_mask(mask):
    data = cc.Data()
    data.add_data(mask.astype(int), "binary")
    orient = data.binary_img.orientation
    data = data.rotate(orient)

    cell = cc.Cell(
        data,
    )
    cell.optimize()
    return cell


def get_cells_from_mask_stack(mask_stack):
    cell_stack = [get_cell_from_mask(mask) for mask in mask_stack]
    return cell_stack

In [ ]:
mask_stack = get_isolated_cells(example_kymo[0])

In [ ]:
mask_stack[0].shape

In [ ]:
plt.imshow(mask_stack[0])

In [ ]:
cell_stack = get_cells_from_mask_stack(mask_stack)

In [ ]:
cell_stack[1].length

In [ ]:
import paulssonlab.deaton.trenchripper.trenchripper as tr

In [ ]:
handle = tr.pandas_hdf5_handler(
    "/home/de64/scratch/de64/sync_folder/2021-06-14_lDE20_biofloat_fullrun_1/Growth_Division/metadata.hdf5"
)

In [ ]:
data = cc.Data()
data.add_data(example_kymo, "binary")

In [ ]:
cell_list = cc.data_to_cells(
    data,
    initial_crop=1,
    final_crop=0,
    remove_bordering=False,
    remove_multiple_cells=False,
)

In [ ]:
cell_list

In [ ]:
clp = cc.CellListPlot(cell_list)

In [ ]:
fig, axes = plt.subplots(2, 2)
clp.hist_property(ax=axes[0, 0], prop="surface")
clp.hist_property(ax=axes[0, 1], prop="length")
clp.hist_property(ax=axes[1, 0], prop="area")
clp.hist_property(ax=axes[1, 1], prop="volume")
plt.tight_layout()
plt.autoscale()

In [ ]:
cell = cc.Cell(
    data,
)
cp = cc.CellPlot(cell)

In [ ]:
plt.imshow(example_label)

In [ ]:
rps = sk.measure.regionprops(example_label)

In [ ]:
rps[0].orientation

In [ ]:
bounding_box = rps[0].bbox
centroid = rps[0].centroid
bottom, top = bounding_box[0] - 2, bounding_box[2] + 2
left, right = bounding_box[1] - 2, bounding_box[3] + 2
orientation = rps[0].orientation
major_slope = np.tan((np.pi / 2) - orientation)
minor_slope = -np.tan(orientation)

In [ ]:
top_x = ((top - centroid[0]) / major_slope) + centroid[1]
bottom_x = ((bottom - centroid[0]) / major_slope) + centroid[1]

left_y = ((left - centroid[1]) * minor_slope) + centroid[0]
right_y = ((right - centroid[1]) * minor_slope) + centroid[0]

length = np.sum(
    sk.measure.profile_line(
        example_label == 1, (bottom, bottom_x), (top, top_x), mode="constant"
    )
    > 0.5
)
width = np.sum(
    sk.measure.profile_line(
        example_label == 1, (left_y, left), (right_y, right), mode="constant"
    )
    > 0.5
)

In [ ]:
plt.imshow(example_label == 1)

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(example_mask ^ sk.morphology.binary_erosion(example_mask))
plt.show()
plt.figure(figsize=(10, 10))
plt.imshow(
    sk.morphology.binary_erosion(example_mask)
    ^ sk.morphology.binary_dilation(example_mask)
)
plt.show()

In [ ]:
plt.figure(figsize=(20, 20))
coords = sk.measure.find_contours(example_mask, level=0)[0]
plt.imshow(example_mask)
plt.plot(coords[:, 1], coords[:, 0], "-r", linewidth=2)
plt.show()

In [ ]:
moo = np.stack([coords[coords[:, 1] > 7][:, 1], coords[coords[:, 1] > 7][:, 0]], axis=1)
moo[:, 0] = moo[:, 0] - 7.0

In [ ]:
moo

In [ ]:
import trimesh

In [ ]:
test = trimesh.creation.revolve(
    moo,
    angle=2 * np.pi,
)

In [ ]:
test.area

In [ ]:
test.show()

In [ ]:
for contour in find_contours(img, 0):
    coords = approximate_polygon(contour, tolerance=2.5)
    ax2.plot(coords[:, 1], coords[:, 0], "-r", linewidth=2)
    coords2 = approximate_polygon(contour, tolerance=39.5)
    ax2.plot(coords2[:, 1], coords2[:, 0], "-g", linewidth=2)
    print("Number of coordinates:", len(contour), len(coords), len(coords2))